## Coffee Market Visualization - DRAFT
Includes thought process and trial and error while exploring the data

In [278]:
# Import Libaries
import pandas as pd
import numpy as np
import plotly.express as px
import sqlite3
import plotly.graph_objects as go

### Data Exploration

In [3]:
# Connect to database
con = sqlite3.connect('coffee.db')
cur = con.cursor()

# Read all table names. We find four tables: "market", "price", "SND", and "coffee_type"
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
# print(table_list)

# Read table into dataframe, save tables as df for future use
market_df = pd.read_sql_query("SELECT * FROM market", con)
price_df = pd.read_sql_query("SELECT * FROM price", con)
SND_df = pd.read_sql_query("SELECT * FROM SND", con)
coffee_type_df = pd.read_sql_query("SELECT * FROM coffee_type", con)

In [4]:
# General Research Questions:

# How many types of coffee in this dataframe?
coffee_type_df['coffee_type'].unique()
## Found 2 unique types: Arabica and Robusta

# Range of dates?
min(market_df['date']), max(market_df['date'])
## Found 2001-12-31 to 2026-01-31

('2001-12-31', '2026-01-31')

### Basic Data Visualization

In [5]:
## Supply and Demand Graph
# Note: coffee_type_id = 1 is Arabica, coffee_type_id = 2 is Robusta
px.line(SND_df[SND_df["coffee_type_id"] == 1], x = 'date', y = ['total_supply', 'total_demand'], title = 'Supply vs Demand of Arabica Coffee')

In [6]:
px.line(SND_df[SND_df["coffee_type_id"] == 2], x = 'date', y = ['total_supply', 'total_demand'], title = 'Supply vs Demand of Robusta Coffee')

In [7]:
# Why increase supply if demand is stagnant?


In [8]:
# Market Research: Dollar Index and USD/BRL Exchange Rate
px.line(market_df, x = 'date', y = ['DXY'], title = '')
px.line(market_df, x = 'date', y = ['USDBRL'], title = '')


In [9]:
SND_df.groupby('date', as_index = False).mean()

,date,coffee_type_id,certified_stock,total_consumption,total_stock,total_supply,total_demand
0,2001-12-31,1.0,1114.557832,4792.057525,4459.483456,1568.443583,785.610628
1,2002-01-31,1.0,1087.076160,4861.908518,4558.069021,1594.259096,744.775122
2,2002-02-28,1.0,1026.468713,4349.975139,4406.522032,1567.154395,769.710533
3,2002-03-31,1.0,961.625592,4647.049927,4120.613079,1518.538160,806.831154
4,2002-04-30,1.0,892.695043,4755.849295,3905.861783,1479.966651,806.272717
...,...,...,...,...,...,...,...
285,2025-09-30,1.0,1.380259,6263.501538,5917.246701,6723.557093,893.831826
286,2025-10-31,1.0,1.380259,6105.574717,5888.676492,6788.702105,937.483595
287,2025-11-30,1.0,1.380259,6449.230063,5911.867494,6863.617168,937.811736
288,2025-12-31,1.0,1.380259,6234.823354,5932.597473,6934.947852,944.966686


In [10]:
px.line(SND_df.groupby('date', as_index=False).mean(), x = 'date', y = ['certified_stock', 'total_stock', 'total_consumption'], title = '')

In [11]:
# Note: Arabica and Robusta represent id 1 and id 2
px.line(price_df, x = 'date', y = ['price'], color = "coffee_type_id", title = '')

In [12]:
# Merge the dataframes for ease of analysis. Can do this b/c small dataset, but not recommended for large datasets

df = SND_df.merge(coffee_type_df, on = ['coffee_type_id'])
df = df.merge(market_df, on = ['date'])
df = df.merge(price_df, on = ['date', 'coffee_type_id'])

df

,date,coffee_type_id,certified_stock,total_consumption,total_stock,total_supply,total_demand,coffee_type,actvsproj_x,DXY,USDBRL,actvsproj_y,price
0,2001-12-31,1,1114.557832,4792.057525,4459.483456,1568.443583,785.610628,Arabica,act,116.750000,2.311000,act,48.150000
1,2002-01-31,1,1087.076160,4861.908518,4558.069021,1594.259096,744.775122,Arabica,act,120.239998,2.413000,act,47.650000
2,2002-02-28,1,1026.468713,4349.975139,4406.522032,1567.154395,769.710533,Arabica,act,119.160004,2.365000,act,46.300000
3,2002-03-31,1,961.625592,4647.049927,4120.613079,1518.538160,806.831154,Arabica,act,118.620003,2.325000,act,59.200000
4,2002-04-30,1,892.695043,4755.849295,3905.861783,1479.966651,806.272717,Arabica,act,115.190002,2.361000,act,52.850000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,2025-09-30,1,1.380259,6263.501538,5917.246701,6723.557093,893.831826,Arabica,proj,104.270060,5.836315,proj,154.800000
489,2025-10-31,1,1.380259,6105.574717,5888.676492,6788.702105,937.483595,Arabica,proj,104.193267,5.871527,proj,155.000000
490,2025-11-30,1,1.380259,6449.230063,5911.867494,6863.617168,937.811736,Arabica,proj,104.116531,5.906739,proj,155.200000
491,2025-12-31,1,1.380259,6234.823354,5932.597473,6934.947852,944.966686,Arabica,proj,104.039851,5.941951,proj,155.400000


In [13]:
px.line(df[df["coffee_type"] == "Robusta"], x = 'date', y = ['total_supply', 'total_demand'], color = "actvsproj_x", title = 'Supply vs Demand of Arabica Coffee')

In [301]:
# Playing with Dash

import dash
from dash import Dash, html, dcc, dash_table
from dash.dependencies import Input, Output

app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div(id = "output_container", children = []),

    dcc.Dropdown(id="select_coffee_type",
                 options=[
                     {"label": "All", "value": 'All'},
                     {"label": "Arabica", "value": 'Arabica'},
                     {"label": "Robusta", "value": 'Robusta'}],
                 multi=False,
                 value='All',
                 style={'width': "40%"}
                 ),

    dcc.Graph(id="supply_demand_graph", figure = {})
])

@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='supply_demand_graph', component_property='figure')],
    [Input(component_id='select_coffee_type', component_property='value')]
)
def update_graph(option_slctd):
    print(option_slctd)
    print(type(option_slctd))

    container = "The coffee type chosen by user was: {}".format(option_slctd)

    dff = df.copy()
    dff = dff[dff["coffee_type"] == option_slctd]

    # Plotly Express
    fig = px.line(dff, 
        x = 'date', 
        y = ['total_supply', 'total_demand'], 
        color = "actvsproj_x", 
        title = 'Supply vs Demand of %s Coffee' % option_slctd
    )

    return container, fig

app.run()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050
Press CTRL+C to quit


In [15]:
df_month = df.copy()
df_month['date'] = pd.to_datetime(df_month.date)
df_month = df_month.groupby([df_month.date.dt.to_period('Y'), df_month.coffee_type_id]).mean('total_demand').reset_index()
df_month['date'] = df_month['date'].astype(str)

In [35]:
fig = px.histogram(df_month, x = 'date', color = 'coffee_type_id', y = 'price', title = 'Average Supply of Coffee by Year', barmode='group')

fig.show()

In [55]:
fig = px.histogram(df_month[df_month['coffee_type_id'] == 2], x = 'date', y = ['price'], title = 'Average Supply of Coffee by Year', barmode = 'group')
fig

In [56]:
df

,date,coffee_type_id,certified_stock,total_consumption,total_stock,total_supply,total_demand,coffee_type,actvsproj_x,DXY,USDBRL,actvsproj_y,price
0,2001-12-31,1,1114.557832,4792.057525,4459.483456,1568.443583,785.610628,Arabica,act,116.750000,2.311000,act,48.150000
1,2002-01-31,1,1087.076160,4861.908518,4558.069021,1594.259096,744.775122,Arabica,act,120.239998,2.413000,act,47.650000
2,2002-02-28,1,1026.468713,4349.975139,4406.522032,1567.154395,769.710533,Arabica,act,119.160004,2.365000,act,46.300000
3,2002-03-31,1,961.625592,4647.049927,4120.613079,1518.538160,806.831154,Arabica,act,118.620003,2.325000,act,59.200000
4,2002-04-30,1,892.695043,4755.849295,3905.861783,1479.966651,806.272717,Arabica,act,115.190002,2.361000,act,52.850000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,2025-09-30,1,1.380259,6263.501538,5917.246701,6723.557093,893.831826,Arabica,proj,104.270060,5.836315,proj,154.800000
489,2025-10-31,1,1.380259,6105.574717,5888.676492,6788.702105,937.483595,Arabica,proj,104.193267,5.871527,proj,155.000000
490,2025-11-30,1,1.380259,6449.230063,5911.867494,6863.617168,937.811736,Arabica,proj,104.116531,5.906739,proj,155.200000
491,2025-12-31,1,1.380259,6234.823354,5932.597473,6934.947852,944.966686,Arabica,proj,104.039851,5.941951,proj,155.400000


In [61]:
df.groupby('coffee_type_id')['certified_stock'].transform(lambda x: x.rolling(12, 12).mean())

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
         ...    
488    12.611992
489     9.488987
490     6.872140
491     4.761450
492     3.156917
Name: certified_stock, Length: 493, dtype: float64

In [349]:
# Dash Visualization - Run all code and go to the provided link to see the visualization

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1(children='Select Coffee Type'),
    dcc.Dropdown(id="select_coffee_type",
                 options=[
                     {"label": "Arabica", "value": 'Arabica'},
                     {"label": "Robusta", "value": 'Robusta'}],
                 multi=False,
                 value='Arabica',
                 style={'width': "40%"}
                 ),

    html.H1(children='Interactive Graph'),
    html.H3(children='Select Metric(s)'),
    dcc.Checklist(id="select_metric",
                options=[
                    {"label": "Price", "value": 'price'},
                    {"label": "Supply", "value": 'total_supply'},
                    {"label": "Demand", "value": 'total_demand'},
                    {"label": "Consumption", "value": 'total_consumption'},
                    {"label": "Stock", "value": 'total_stock'},
                    {"label": "Certified Stock", "value": 'certified_stock'}],
                style={'width': "40%"}
                ),

    html.H3(children='Output: Metrics Over Time'),
    dcc.Graph(id="supply_demand_graph", figure = {}),
    
    html.H1(children='Key Metrics'),

    html.H3(children='Select Year'),
    dcc.Slider(2002, 2023, 1,
               value = 2023,
               marks=None,
               tooltip={"placement": "bottom", "always_visible": True},
               id = "selected_year"),

    html.H3(children='Output: Market Evolution of Selected Coffee Type to Previous Year'),
    dash_table.DataTable(
            id="output_df",
            style_as_list_view=True,
        ),
])

@app.callback(
    [Output(component_id='supply_demand_graph', component_property='figure'),
     Output(component_id='output_df', component_property='data'),
     Output(component_id='output_df', component_property='columns')],
    [Input(component_id='select_coffee_type', component_property='value'),
    Input(component_id='select_metric', component_property='value'),
    Input(component_id='selected_year', component_property='value')]
)
def update_graph(coffee_type, metric, year):
    container = metric
    year = str(year)

    # Clone df to dff
    dff = df.copy()
    dff = dff[dff["coffee_type"] == coffee_type]

    # For highlighting area of projections
    projs = dff[dff['actvsproj_x'] == 'proj']['date']

    # Main figure
    fig = px.line(dff, x = 'date', y = metric)
    fig.add_vrect(
    x0=min(projs), x1=max(projs),
    fillcolor="LightSalmon", opacity=0.5,
    layer="below", line_width=0,
    ),

    # Table
    # New df
    df_year = dff.copy()
    df_year['date'] = pd.to_datetime(df_year.date)
    df_year = df_year.groupby([df_year.date.dt.to_period('Y'), df_year.coffee_type_id]).mean('total_demand').reset_index()
    df_year['date'] = df_year['date'].astype(str)
    df_year = df_year[df_year['date'].isin([str(int(year) - 1), year])]

    print(df_year)
    # metrics df, gives percent change of year over year
    # get year data in a list
    names = df_year.columns.tolist()[2:]
    prev_year_data = df_year.iloc[0].tolist()[2:]
    year_data = df_year.iloc[1].tolist()[2:]
    # percent change
    pctch = np.divide(np.subtract(year_data, prev_year_data), prev_year_data) * 100

    # create new df
    df_changes = pd.DataFrame({'Metric': names, str(int(year) - 1): prev_year_data, year: year_data, "% Change": pctch})
    df_changes = df_changes.round(2)
    df_changes['% Change'] = df_changes['% Change'].astype(str) + '%'
    

    data=df_changes.to_dict('records')
    columns=[{"name": i, "id": i} for i in df_changes.columns]

    return fig, data, columns

app.run()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [05/Nov/2023 03:44:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2023 03:44:46] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2023 03:44:46] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2023 03:44:46] "GET /_favicon.ico?v=2.7.0 HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2023 03:44:46] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Nov/2023 03:44:46] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Nov/2023 03:44:46] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Nov/2023 03:44:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2023 03:44:46] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Nov/2023 03:44:46] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Nov/

    date  coffee_type_id  certified_stock  total_consumption  total_stock  \
21  2022               1       310.203500        5778.516359  5266.628373   
22  2023               1       214.956705        5732.940436  5439.942186   

    total_supply  total_demand         DXY    USDBRL       price  
21   4167.635809    841.407688  104.056666  5.122400  212.583333  
22   4937.126664    818.612838  103.985375  5.039835  163.875000  


127.0.0.1 - - [05/Nov/2023 03:44:48] "POST /_dash-update-component HTTP/1.1" 200 -


    date  coffee_type_id  certified_stock  total_consumption  total_stock  \
14  2022               2      3136.344131        5299.135424  4233.026637   
15  2023               2      1988.266903        5387.314082  3800.671298   

    total_supply  total_demand         DXY    USDBRL        price  
14   1636.723261    774.161600  104.056666  5.122400  2050.083333  
15   1970.822444    794.226054  103.985375  5.039835  2322.833333  


...

    date  coffee_type_id  certified_stock  total_consumption  total_stock  \
21  2022               1       310.203500        5778.516359  5266.628373   
22  2023               1       214.956705        5732.940436  5439.942186   

    total_supply  total_demand         DXY    USDBRL       price  
21   4167.635809    841.407688  104.056666  5.122400  212.583333  
22   4937.126664    818.612838  103.985375  5.039835  163.875000  


127.0.0.1 - - [05/Nov/2023 03:45:02] "POST /_dash-update-component HTTP/1.1" 200 -


    date  coffee_type_id  certified_stock  total_consumption  total_stock  \
13  2014               1       948.844871        5610.724150  5155.657075   
14  2015               1       802.609936        5736.742362  5063.377808   

    total_supply  total_demand        DXY    USDBRL       price  
13   2437.372849   1030.549154  83.037499  2.363258  182.900000  
14   2428.141434   1059.003336  96.735833  3.390858  132.466667  


127.0.0.1 - - [05/Nov/2023 03:45:05] "POST /_dash-update-component HTTP/1.1" 200 -


    date  coffee_type_id  certified_stock  total_consumption  total_stock  \
21  2022               1       310.203500        5778.516359  5266.628373   
22  2023               1       214.956705        5732.940436  5439.942186   

    total_supply  total_demand         DXY    USDBRL       price  
21   4167.635809    841.407688  104.056666  5.122400  212.583333  
22   4937.126664    818.612838  103.985375  5.039835  163.875000  


In [341]:
year = '2023'
dff = df.copy()
dff = dff[dff["coffee_type"] == "Arabica"]

# New df
df_year = dff.copy()
df_year['date'] = pd.to_datetime(df_year.date)
df_year = df_year.groupby([df_year.date.dt.to_period('Y'), df_year.coffee_type_id]).mean('total_demand').reset_index()
df_year['date'] = df_year['date'].astype(str)
df_year = df_year[df_year['date'].isin([str(int(year) - 1), year])]

print(df_year)
# metrics df, gives percent change of year over year
# get year data in a list
names = df_year.columns.tolist()[2:]
prev_year_data = df_year.iloc[0].tolist()[2:]
year_data = df_year.iloc[1].tolist()[2:]
# percent change
pctch = np.divide(np.subtract(year_data, prev_year_data), prev_year_data) * 100

# create new df
df_changes = pd.DataFrame({'Metric': names, str(int(year) - 1): prev_year_data, year: year_data, "% Change": pctch})
df_changes = df_changes.round(2)
df_changes['% Change'] = df_changes['% Change'].astype(str) + '%'
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_changes.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=df_changes.transpose().values.tolist(),
               fill_color='lavender',
               align='left'))
])

fig.show()

    date  coffee_type_id  certified_stock  total_consumption  total_stock  \
21  2022               1       310.203500        5778.516359  5266.628373   
22  2023               1       214.956705        5732.940436  5439.942186   

    total_supply  total_demand         DXY    USDBRL       price  
21   4167.635809    841.407688  104.056666  5.122400  212.583333  
22   4937.126664    818.612838  103.985375  5.039835  163.875000  


In [294]:
dff_changes = dff_changes.round(2)
dff_changes['% Change'] = dff_changes['% Change'].astype(str) + '%'
fig = go.Figure(data=[go.Table(
    header=dict(values=list(dff_changes.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=dff_changes.transpose().values.tolist(),
               fill_color='lavender',
               align='left'))
])

fig.show()

In [210]:
str(int(year) - 1)

'2022'